In [ ]:
!pip install scanpy
!pip install scikit-misc
!pip install anndata
!pip install pyro-ppl
!pip install pytorch-forecasting
!pip install scvi-tools

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 989.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 3.0 MB/s eta 0:00:00
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8023 sha256=c47a9e80bc62f104a62fd7648cca8f2e4d65f19779ac306a49929e82d854feb1
  Stored in directory: /root/.cache/pip/wheels/6a/aa/b9/eb5d4031476ec10802795b97ccf937b9bd998d68a9b268765a
Successfully built session-info
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#import drive.MyDrive.CSE527.GeneDataset as gd, custom dataloader is not necessary for these tasks and slightly slower than annloader
import anndata
import scanpy as sc
import torch
import model as m
import network_utils as n
import scvi
import numpy as np
from anndata.experimental import AnnLoader
from importlib import reload
from sklearn.preprocessing import LabelEncoder

In [ ]:
adata = sc.read_h5ad('drive/MyDrive/CSE527/data_chunk2.h5ad')
adata = adata[:,adata.var.sort_values(by="Gene").index]
aloader = anndata.experimental.AnnLoader(adata,batch_size=1024,shuffle=True)

In [ ]:
enc_lin = [512,256,64]
dec_lin = [512,256,64]
soi = [[40,500,1487,4],[40,100,500,4],[40,20,100,4],[20,5,40,1]]
auto_transformer = m.TransformerAutoencoder(input_size=59480, soih=soi,enc_lin = enc_lin, dec_lin = dec_lin, lr = 0.005, loss="ZINB")

In [ ]:
PATH = "trained_auto_transformerZINB9"
auto_transformer.load_state_dict(torch.load(PATH, weights_only=True))

<All keys matched successfully>

In [ ]:
auto_transformer = auto_transformer.to('cuda')

In [ ]:
bdata = sc.read_h5ad('gridchunk1_1_2.h5ad')
bdata = bdata[:,bdata.var.sort_values(by="Gene").index]
bloader = anndata.experimental.AnnLoader(bdata,batch_size=256,shuffle=True)

In [ ]:
n.train(aloader,auto_transformer, 10, loss_type = "ZINB",start_epoch=9)

Batch  1  Loss  3.126366376876831
Batch  2  Loss  2.2696211338043213
Batch  3  Loss  1.9834163188934326
Batch  4  Loss  1.9504767656326294
Batch  5  Loss  1.8178988695144653
Batch  6  Loss  1.9236786365509033
Batch  7  Loss  1.7570282220840454
Batch  8  Loss  1.8179502487182617
Batch  9  Loss  1.8100172281265259
Batch  10  Loss  1.7045681476593018
Batch  11  Loss  1.6461560726165771
Batch  12  Loss  1.6430808305740356
Batch  13  Loss  1.5820934772491455
Batch  14  Loss  1.4656087160110474
Batch  15  Loss  1.4909175634384155
Batch  16  Loss  1.4435691833496094
Batch  17  Loss  1.3917804956436157
Batch  18  Loss  1.359212040901184
Batch  19  Loss  1.3349696397781372
Batch  20  Loss  1.2690670490264893
Batch  21  Loss  1.338834285736084
Batch  22  Loss  1.217712640762329
Batch  23  Loss  1.2166188955307007
Batch  24  Loss  1.203122615814209
Batch  25  Loss  1.1267715692520142
Batch  26  Loss  1.1636011600494385
Batch  27  Loss  1.1095136404037476
Batch  28  Loss  1.0488907098770142
Batch 

In [ ]:
bdata = bdata.copy()
scvi.model.SCVI.setup_anndata(bdata)
scvi = scvi.model.SCVI(bdata,n_latent = 64)
scvi.train(max_epochs=10)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Training:   0%|          | 0/10 [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=10` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
cdata = sc.read_h5ad('drive/MyDrive/CSE527/gridchunk1_1_3.h5ad')
cdata = cdata[:,cdata.var.sort_values(by="Gene").index]
cdata = cdata.copy()
scvi.model.SCVI.setup_anndata(cdata)
cdata.obsm['X_scvi'] = scvi.get_latent_representation(cdata)
cloader = AnnLoader(cdata, batch_size=256)

INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


In [ ]:
ddata = sc.read_h5ad('drive/MyDrive/CSE527/gridchunk1_1_1.h5ad')
ddata = ddata[:,ddata.var.sort_values(by="Gene").index]
ddata = ddata.copy()
scvi.model.SCVI.setup_anndata(ddata)
ddata.obsm['X_scvi'] = scvi.get_latent_representation(ddata)
dloader = AnnLoader(cdata, batch_size=256)

INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


In [ ]:
model_parameters = filter(lambda p: p.requires_grad, auto_transformer.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])

In [ ]:
params

123522652

In [ ]:
enc_lin = [512,256,64]
dec_lin = [512,256,64]
soi = [[40,1000,1487,4],[40,500,1000,4],[40,100,500,4],[40,20,100,4]]
auto_transformer = m.TransformerAutoencoder(input_size=59480, soih=soi,enc_lin = enc_lin, dec_lin = dec_lin, lr = 0.003, loss="ZINB")
auto_transformer = auto_transformer.to('cuda')

In [ ]:
auto_basic = m.AutoEncoder([0]*59480)
auto_basic.to('cuda')

AutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=59480, out_features=2048, bias=True)
    (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=2048, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): LeakyReLU(negative_slope=0.01)
  )
  (decoder): Sequential(
    (0): Linear(in_features=64, out_features=2048, bias=True)
    (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Linear(in_features=2048, out_features=59480, bias=True)
    (4): BatchNorm1d(59480, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
)

In [ ]:

encoder = LabelEncoder()
encoder.fit(bdata.obs["supercluster_term"])
classifier = m.BasicNeuralNetwork(64*[0], bdata.obs["supercluster_term"],encoder,[256,256], lr = 0.005)

In [ ]:
classifier2 = m.BasicNeuralNetwork(64*[0], bdata.obs["supercluster_term"],encoder,[256,256], lr = 0.005)

In [ ]:
classifier2= classifier2.to('cuda')

In [ ]:
m.train(cloader,classifier, 10, loss_type = "cell_type")

Batch  1  Loss  3.154768466949463
Batch  2  Loss  2.361210346221924
Batch  3  Loss  1.8765997886657715
Batch  4  Loss  1.755313515663147
Batch  5  Loss  1.5995681285858154
Batch  6  Loss  1.4947258234024048
Batch  7  Loss  1.497475504875183
Batch  8  Loss  1.454055666923523
Batch  9  Loss  1.2367526292800903
Batch  10  Loss  1.1735377311706543
Batch  11  Loss  1.0381810665130615
Batch  12  Loss  1.1280617713928223
Batch  13  Loss  1.0826324224472046
Batch  14  Loss  0.8808279633522034
Batch  15  Loss  0.7703836560249329
Batch  16  Loss  0.7882378101348877
Batch  17  Loss  0.8499500751495361
Batch  18  Loss  0.9063591957092285
Batch  19  Loss  0.7188360691070557
Batch  20  Loss  0.6936371326446533
Batch  21  Loss  0.7297769784927368
Batch  22  Loss  0.7734454870223999
Batch  23  Loss  0.7297844886779785
Batch  24  Loss  0.6971701979637146
Batch  25  Loss  0.7129335999488831
Batch  26  Loss  0.7393630146980286
Batch  27  Loss  0.7111684083938599
Batch  28  Loss  0.6828168630599976
Batch 

In [ ]:
m.train(bloader,classifier, 10, loss_type = "cell_type",encoder = auto_transformer)

Batch  1  Loss  10.171245574951172
Batch  2  Loss  8.921320915222168
Batch  3  Loss  7.898580551147461
Batch  4  Loss  6.782512664794922
Batch  5  Loss  5.777307510375977
Batch  6  Loss  5.289464950561523
Batch  7  Loss  5.112739562988281
Batch  8  Loss  4.867040634155273
Batch  9  Loss  4.581319332122803
Batch  10  Loss  4.442652225494385
Batch  11  Loss  4.411296367645264
Batch  12  Loss  3.909832000732422
Batch  13  Loss  3.630112886428833
Batch  14  Loss  3.8357112407684326
Batch  15  Loss  3.5742313861846924
Batch  16  Loss  3.5355446338653564
Batch  17  Loss  3.6334190368652344
Batch  18  Loss  3.229795217514038
Batch  19  Loss  3.565889358520508
Batch  20  Loss  3.3743605613708496
Batch  21  Loss  3.054731607437134
Batch  22  Loss  3.2558841705322266
Batch  23  Loss  3.0915355682373047
Batch  24  Loss  3.284111738204956
Batch  25  Loss  3.107218027114868
Batch  26  Loss  3.2896625995635986
Batch  27  Loss  3.0534558296203613
Batch  28  Loss  3.015115737915039
Batch  29  Loss  2.

In [28]:
with torch.no_grad():
    acc = 0
    val_acc = 0
    for batch in bloader:
        #f = batch.obsm['X_scvi'].float().to('cuda')
        f = auto_transformer(batch.X.float().to('cuda'))
        acc += f[:,:,2].sum()/(59480*256*len(bloader))
        #l = torch.tensor(classifier2.encoder.transform(batch.obs['supercluster_term'])).to('cuda')
        #batch_acc = classifier.predict_acc(f,l)
        #print(batch_acc)
        #val_acc += batch_acc/len(cloader)
print(acc)

tensor(-2.3294, device='cuda:0')


In [ ]:
with torch.no_grad():
    acc = 0
    val_acc = 0
    for batch in cloader:
        f = auto_transformer.encode(batch.X.float().to('cuda'))
        out = auto_transformer.decode(f)
        print(out.shape)
        print(out[:,:,0].sum())
        l = torch.tensor(classifier.encoder.transform(batch.obs['supercluster_term']))
        batch_acc = classifier.predict_acc(f,l)
        print(batch_acc)
        val_acc += batch_acc/len(cloader)

torch.Size([256, 59480, 3])
tensor(1.4380e+08, device='cuda:0')
0.0859375
torch.Size([256, 59480, 3])
tensor(1.4322e+08, device='cuda:0')
0.0390625
torch.Size([256, 59480, 3])
tensor(1.4397e+08, device='cuda:0')
0.06640625
torch.Size([256, 59480, 3])
tensor(1.4340e+08, device='cuda:0')
0.0625
torch.Size([256, 59480, 3])
tensor(1.4387e+08, device='cuda:0')
0.06640625
torch.Size([256, 59480, 3])
tensor(1.4359e+08, device='cuda:0')
0.05859375
torch.Size([256, 59480, 3])
tensor(1.4356e+08, device='cuda:0')
0.078125
torch.Size([256, 59480, 3])
tensor(1.4337e+08, device='cuda:0')
0.078125
torch.Size([256, 59480, 3])
tensor(1.4405e+08, device='cuda:0')
0.046875
torch.Size([256, 59480, 3])
tensor(1.4424e+08, device='cuda:0')
0.09765625
torch.Size([256, 59480, 3])
tensor(1.4389e+08, device='cuda:0')
0.109375
torch.Size([256, 59480, 3])
tensor(1.4404e+08, device='cuda:0')
0.06640625
torch.Size([256, 59480, 3])
tensor(1.4387e+08, device='cuda:0')
0.06640625
torch.Size([256, 59480, 3])
tensor(1.43

In [ ]:
val_acc

0.3874627976190476

In [ ]:
# MSE Autotransformer with basic classifier network, test_acc for cell_type classificiation 0.22247023809523803
# ZINB Autotransformer with basic classifier network, test_acc for cell_type classificiation 0.3874627976190476